In [9]:
import numpy as np
import pandas as pd
import os
import torch
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.io import read_image
from torchvision.transforms import functional
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, auc, precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import label_binarize

In [10]:
path = 'dataset_lab3/labels.csv'
df = pd.read_csv(path)
df.head(20)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [11]:
df['breed'].describe()

count                  10222
unique                   120
top       scottish_deerhound
freq                     126
Name: breed, dtype: object

In [12]:
df['breed'] = pd.factorize(df.breed)[0]
df.head(20)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,0
1,001513dfcb2ffafc82cccf4d8bbaba97,1
2,001cdf01b096e06d78e9e5112d419397,2
3,00214f311d5d2247d5dfe4fe24b2303d,3
4,0021f9ceb3235effd7fcde7f7538ed62,4
5,002211c81b498ef88e1b40b9abf84e1d,5
6,00290d3e1fdd27226ba27a8ce248ce85,5
7,002a283a315af96eaea0e28e7163b21b,6
8,003df8b8a8b05244b1d920bb6cf451f9,7
9,0042188c895a2f14ef64a918ed9c7b64,8


In [13]:
class Dogs(Dataset):

    def __init__(self, path, annotation, transform = None, target_transform = None):
        self.img_labels = annotation['id']
        self.img_class = annotation['breed']
        self.data_dir = path
        self.img_dir = annotation['id']
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = str(self.data_dir + self.img_dir[idx] + '.jpg')
        image = read_image(img_path)
        image = functional.to_pil_image(image)
        label = self.img_class.loc[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [19]:
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
path = 'dataset_lab3/train/'
train, test = train_test_split(df, test_size=0.05, random_state=42,shuffle=True, stratify=df['breed'])

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

train_ds = Dogs(path, train, transform)
train_dl = DataLoader(train_ds, batch_size=100, shuffle=True, drop_last=True)
test_ds = Dogs(path, test, transform)
test_dl = DataLoader(test_ds, batch_size=100, shuffle=True, drop_last=True)

In [20]:
device_count = torch.cuda.device_count()
print(f"Доступно устройств GPU: {device_count}")

for i in range(device_count):
    device_name = torch.cuda.get_device_name(i)
    print(f"Устройство {i}: {device_name}")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

Доступно устройств GPU: 0
cpu


In [21]:
class Dogs_NN(torch.nn.Module):
    def __init__(self):
        super(Dogs_NN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=1)
        # 6*222*222
        self.pool1 = nn.MaxPool2d(2, stride=2)
        # 6*111*111
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5, stride=2, padding=1)
        # 12*55*55
        self.linear1 = nn.Linear(12*55*55, 1000)
        self.linear2 = nn.Linear(1000, 120)
        #Сворачиваем до 120

    def forward(self, input):
        output = F.relu(self.conv1(input))
        output = self.pool1(output)
        output = F.relu(self.conv2(output))
        output = output.view(output.size(0), 12*55*55)
        output = F.relu(self.linear1(output))
        output = self.linear2(output)

        return output


In [ ]:
timing = time.time()
my_nn = Dogs_NN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(my_nn.parameters(), lr=0.001)

train_losses = []
valid_losses = []
num_epochs = 5 

for epoch in range(num_epochs):
    train_loss = 0.0
    for batch, labels in train_dl:
        batch, labels = batch.to(device), labels.to(device)
        my_nn.train()
        batch = batch.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = my_nn(batch.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss = {loss.item():.4f}')
    my_nn.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for batch, labels in test_dl:
            batch, labels = batch.to(device), labels.to(device)
            outputs = my_nn(batch.float())
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    print(f'Valid loss = {loss.item():.4f}')

    train_losses.append(train_loss / len(train_dl))
    valid_losses.append(valid_loss / len(test_dl))
    print("time of epoch:", time.time() - timing)
    timing = time.time()

print('Training is finished')

torch.save(my_nn.state_dict(), 'Dogs_classifier.pth')
combined_list = list(zip(train_losses, valid_losses))
losses = pd.DataFrame(combined_list, columns=['train_loss', 'valid_loss'])
losses.to_csv('Losses_5_epoch.csv', index=False)

plt.figure(figsize = (9, 6))
colors = ['#00DD99', '#9900DD']
sns.lineplot(data=losses, palette = colors)
plt.grid(ls = ':')
plt.show()

Epoch [1/5], Loss = 4.7801
Valid loss = 4.7503
time of epoch: 151.65383052825928
Epoch [2/5], Loss = 4.6239
Valid loss = 4.7382
time of epoch: 137.031259059906


In [ ]:
class Dogs_NN_2(torch.nn.Module):
    # Изначально 3*224*224
    def __init__(self):
        super(Dogs_NN_2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=1)
        # 6*222*222
        self.pool1 = nn.MaxPool2d(3, stride=2)
        # 6*110*110
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=10, stride=1, padding=1)
        # 12*103*103
        self.pool2 = nn.MaxPool2d(2, stride=3)
        # 12*51*51
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=20, stride=1, padding=1)
        # 24*34*34
        self.pool3 = nn.MaxPool2d(2, stride=3)

        self.linear1 = nn.Linear(24*6*6, 512)
        self.linear2 = nn.Linear(512, 120)
        #Сворачиваем до 120 (сколько целевых классов)

    def forward(self, input):
        output = F.relu(self.conv1(input))
        output = self.pool1(output)
        output = F.relu(self.conv2(output))
        output = self.pool2(output)
        output = F.relu(self.conv3(output))
        output = self.pool3(output)
        output = output.view(output.size(0), 24*6*6)
        output = F.relu(self.linear1(output))
        output = self.linear2(output)

        return output        

In [ ]:
timing = time.time()
my_nn = Dogs_NN_2().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(my_nn.parameters(), lr=0.001)

train_losses = []
valid_losses = []
num_epochs = 5 

for epoch in range(num_epochs):
    train_loss = 0.0
    for batch, labels in train_dl:
        batch, labels = batch.to(device), labels.to(device)
        my_nn.train()
        batch = batch.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = my_nn(batch.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss = {loss.item():.4f}')
    my_nn.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for batch, labels in test_dl:
            batch, labels = batch.to(device), labels.to(device)
            outputs = my_nn(batch.float())
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    print(f'Valid loss = {loss.item():.4f}')

    train_losses.append(train_loss / len(train_dl))
    valid_losses.append(valid_loss / len(test_dl))
    print("time of epoch:", time.time() - timing)
    timing = time.time()
print('Training is finished')

torch.save(my_nn.state_dict(), 'Dogs_classifier_2.pth')
combined_list = list(zip(train_losses, valid_losses))
losses = pd.DataFrame(combined_list, columns=['train_loss', 'valid_loss'])
losses.to_csv('Losses_5_epoch_2.csv', index=False)

plt.figure(figsize = (9, 6))
colors = ['#00DD99', '#9900DD']
sns.lineplot(data=losses, palette = colors)
plt.grid(ls = ':')
plt.show()

In [ ]:
from torchvision.models import shufflenet_v2_x0_5

model = shufflenet_v2_x0_5(pretrained=True)
num_classes = df['breed'].nunique()
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
valid_losses = []
num_epochs = 5

for epoch in range(num_epochs):
    train_loss = 0.0
    for batch, labels in train_dl:
        batch, labels = batch.to(device), labels.to(device)
        model.train()
        optimizer.zero_grad()
        outputs = model(batch.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss = {loss.item():.4f}')
    
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for batch, labels in test_dl:
            batch, labels = batch.to(device), labels.to(device)
            outputs = model(batch.float())
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    print(f'Valid loss = {loss.item():.4f}')
    
    train_losses.append(train_loss / len(train_dl))
    valid_losses.append(valid_loss / len(test_dl))
    
print('Training is finished')

torch.save(model, 'shufflenet_v2_x0_5_Dogs_classifier.pth')
combined_list = list(zip(train_losses, valid_losses))
losses = pd.DataFrame(combined_list, columns=['train_loss', 'valid_loss'])
losses.to_csv('shufflenet_v2_x0_5_Losses_5_epoch.csv', index=False)

f = plt.figure(figsize = (9, 6))
colors = ['#00DD99', '#9900DD']
sns.lineplot(data=losses, palette = colors)
plt.grid(ls = ':')
plt.show()

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights

weights = ResNet18_Weights.DEFAULT
preprocess = weights.transforms()

model = resnet18(pretrained=True)

num_classes = df['breed'].nunique()
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses = []
valid_losses = []
num_epochs = 5

for epoch in range(num_epochs):
    train_loss = 0.0
    for batch, labels in train_dl:
        batch, labels = batch.to(device), labels.to(device)
        model.train()
        optimizer.zero_grad()
        outputs = model(batch.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss = {loss.item():.4f}')
    
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for batch, labels in test_dl:
            batch, labels = batch.to(device), labels.to(device)
            outputs = model(batch.float())
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    print(f'Valid loss = {loss.item():.4f}')
    
    train_losses.append(train_loss / len(train_dl))
    valid_losses.append(valid_loss / len(test_dl))
    
print('Training is finished')

torch.save(model, 'resnet18_Dogs_classifier.pth')
combined_list = list(zip(train_losses, valid_losses))
losses = pd.DataFrame(combined_list, columns=['train_loss', 'valid_loss'])

f = plt.figure(figsize = (9, 6))
colors = ['#00DD99', '#9900DD']
sns.lineplot(data=losses, palette = colors)
plt.grid(ls = ':')
plt.show()

In [ ]:
model = Dogs_NN()
model.load_state_dict(torch.load('Dogs_classifier.pth'))
model.eval() 

transform = Compose([
    Resize((224,224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ])
    
valid_data = Dogs(path, test, transform=transform)
valid_dataloader = DataLoader(valid_data, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

pred_class = []    
with torch.no_grad():
        for batch, labels in valid_dataloader:
            batch, labels = batch.to(device), labels.to(device)
            outputs = model(batch.float())
            _, predicted = torch.max(outputs, 1)
            predicted = predicted.cpu()
            pred_class.append(predicted.item())
            
true_class = test['breed'].tolist()
f1_macro = f1_score(true_class, pred_class, average='macro')
f1_micro = f1_score(true_class, pred_class, average='micro')
f1_weighted = f1_score(true_class, pred_class, average='weighted')
print("F1 метрика (макро):", f1_macro)
print("F1 метрика (микро):", f1_micro)
print("F1 метрика (среднее взвешенное):", f1_weighted)

precision, recall, _ , _ = precision_recall_fscore_support(true_class, pred_class)
accuracy = accuracy_score(true_class, pred_class)
precision_aver = precision.mean()
recall_aver = recall.mean()

print('Accuracy:', accuracy)
print('Average Precision:', precision_aver)
print('Average Recall:', recall_aver)

In [ ]:
model = Dogs_NN_2()
model.load_state_dict(torch.load('Dogs_classifier_2.pth'))
model.eval() 

transform = Compose([
    Resize((224,224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ])
    
valid_data = Dogs(path, test, transform=transform)
valid_dataloader = DataLoader(valid_data, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

pred_class = []    
with torch.no_grad():
        for batch, labels in valid_dataloader:
            batch, labels = batch.to(device), labels.to(device)
            outputs = model(batch.float())
            _, predicted = torch.max(outputs, 1)
            predicted = predicted.cpu()
            pred_class.append(predicted.item())
            
true_class = test['breed'].tolist()
f1_macro = f1_score(true_class, pred_class, average='macro')
f1_micro = f1_score(true_class, pred_class, average='micro')
f1_weighted = f1_score(true_class, pred_class, average='weighted')
print("F1 метрика (макро):", f1_macro)
print("F1 метрика (микро):", f1_micro)
print("F1 метрика (среднее взвешенное):", f1_weighted)

precision, recall, _ , _ = precision_recall_fscore_support(true_class, pred_class)
accuracy = accuracy_score(true_class, pred_class)
precision_aver = precision.mean()
recall_aver = recall.mean()

print('Accuracy:', accuracy)
print('Average Precision:', precision_aver)
print('Average Recall:', recall_aver)

In [17]:
model = torch.load('shufflenet_v2_x0_5_Dogs_classifier.pth')
model.eval()

transform = Compose([
    Resize((224,224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ])
    
valid_data = Dogs(path, test, transform=transform)
valid_dataloader = DataLoader(valid_data, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

pred_class = []    
with torch.no_grad():
        for batch, labels in valid_dataloader:
            batch, labels = batch.to(device), labels.to(device)
            outputs = model(batch.float())
            _, predicted = torch.max(outputs, 1)
            predicted = predicted.cpu()
            pred_class.append(predicted.item())


true_class = test['breed'].tolist()
f1_macro = f1_score(true_class, pred_class, average='macro')
f1_micro = f1_score(true_class, pred_class, average='micro')
print("F1 метрика (макро):", f1_macro)
print("F1 метрика (микро):", f1_micro)
f1_weighted = f1_score(true_class, pred_class, average='weighted')
print("F1 метрика (среднее взвешенное):", f1_weighted)

precision, recall, _ , _ = precision_recall_fscore_support(true_class, pred_class)
accuracy = accuracy_score(true_class, pred_class)
precision_aver = precision.mean()
recall_aver = recall.mean()

print('Accuracy:', accuracy)
print('Average Precision:', precision_aver)
print('Average Recall:', recall_aver)

F1 метрика (макро): 0.4096590479523916
F1 метрика (микро): 0.451171875
F1 метрика (среднее взвешенное): 0.42152491877451964
Accuracy: 0.451171875
Average Precision: 0.4405888787138787
Average Recall: 0.43652777777777774


C:\Users\Acer\PycharmProjects\lab_script\venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model = torch.load('resnet18_Dogs_classifier.pth')
model.eval()

transform = Compose([
    Resize((224,224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ])
    
valid_data = Dogs(path, test, transform=transform)
valid_dataloader = DataLoader(valid_data, batch_size=1, shuffle=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

pred_class = []    

with torch.no_grad():
        for batch, labels in valid_dataloader:
            batch, labels = batch.to(device), labels.to(device)
            outputs = model(batch.float())
            _, predicted = torch.max(outputs, 1)
            predicted = predicted.cpu()
            pred_class.append(predicted.item())

true_class = test['breed'].tolist()
f1_macro = f1_score(true_class, pred_class, average='macro')
f1_micro = f1_score(true_class, pred_class, average='micro')
print("F1 метрика (макро):", f1_macro)
print("F1 метрика (микро):", f1_micro)
f1_weighted = f1_score(true_class, pred_class, average='weighted')
print("F1 метрика (среднее взвешенное):", f1_weighted)

precision, recall, _ , _ = precision_recall_fscore_support(true_class, pred_class)
accuracy = accuracy_score(true_class, pred_class)
precision_aver = precision.mean()
recall_aver = recall.mean()

print('Accuracy:', accuracy)
print('Average Precision:', precision_aver)
print('Average Recall:', recall_aver)